In [ ]:
%pip -q install -q pandas nltk transformers

In [ ]:
import nltk
import pandas as pd
from transformers import pipeline

In [ ]:
nltk.download('punkt')

In [ ]:
df10k = pd.read_csv('dataset_10k.csv')
df10q = pd.read_csv('dataset_10q.csv')

In [ ]:
print(len(df10k))
print(len(df10q))

In [ ]:
summarizer = pipeline('summarization', model='t5-base')

classifier_model_name = 'bhadresh-savani/distilbert-base-uncased-emotion'
classifier_emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

#classifier_model_name = 'ProsusA I/finbert'
#classifier_emotions = ['positive', 'neutral', 'negative']

classifier = pipeline('text-classification', model=classifier_model_name)

In [ ]:
def find_emotional_sentences(text, emotions, threshold):
    sentences_by_emotion = {}
    for e in emotions:
        sentences_by_emotion[e]=[]
    sentences = nltk.sent_tokenize(text)
    print(f'Document has {len(text)} characters and {len(sentences)} sentences.')
    for s in sentences:
        prediction = classifier(s)
        if (prediction[0]['label']!='neutral' and prediction[0]['score']>threshold):
            #print (f'Sentence #{sentences.index(s)}: {prediction} {s}')
            sentences_by_emotion[prediction[0]['label']].append(s)
    for e in emotions:
        print(f'{e}: {len(sentences_by_emotion[e])} sentences')
    return sentences_by_emotion

In [ ]:
def summarize_sentences(sentences_by_emotion, min_length, max_length):
    for k in sentences_by_emotion.keys():
        if (len(sentences_by_emotion[k])!=0):
            text = ' '.join(sentences_by_emotion[k])
            summary = summarizer(text, min_length=min_length, max_length=max_length)
            print(f"{k.upper()}: {summary[0]['summary_text']}\n")

In [ ]:
text = df10q.iloc[1]['Management’s Discussion and Analysis of Financial Condition and Results of Operations']

sentences_by_emotion = find_emotional_sentences(text, classifier_emotions, 0.7)

In [ ]:
sentences_by_emotion['anger']

In [ ]:
summarize_sentences(sentences_by_emotion, min_length=30, max_length=60)